In [ ]:
# Input and output file paths
input_file = "folder_class_mapping.txt"
output_file = "folder_class_mapping_0.txt"

# Read the original file
with open(input_file, "r") as f:
    lines = f.readlines()

new_lines = []
for line in lines:
    # Skip empty lines
    if not line.strip():
        continue

    # Split by tab or whitespace
    parts = line.strip().split()
    try:
        # Subtract 1 from the class ID
        parts[0] = str(int(parts[0]) - 1)
        # Join and add newline
        new_lines.append("\t".join(parts) + "\n")
    except ValueError:
        # If the line cannot be converted to int, skip
        print(f"Skipping invalid line: {line.strip()}")

# Write the corrected 0-based file
with open(output_file, "w") as f:
    f.writelines(new_lines)

print(f"Class IDs converted to 0-based in {output_file}")


In [ ]:
import os
import pandas as pd

# === CONFIG ===
base_dataset_path = r"Dataset_Repository"
mapping_file = r"folder_class_mapping_0.txt"

# === READ MAPPING FILE ===
try:
    mapping_df = pd.read_csv(mapping_file, sep="\t")
except:
    mapping_df = pd.read_csv(mapping_file, sep="\s+", header=None)  # fallback

# Ensure correct column names
if mapping_df.shape[1] == 2:
    mapping_df.columns = ["Class_ID", "Folder"]

# Make sure Class_ID is integer
mapping_df["Class_ID"] = mapping_df["Class_ID"].astype(int)
mapping_dict = dict(zip(mapping_df["Folder"], mapping_df["Class_ID"]))

# === LOOP THROUGH ALL SUBFOLDERS ===
for folder_name in os.listdir(base_dataset_path):
    folder_path = os.path.join(base_dataset_path, folder_name)
    
    if not os.path.isdir(folder_path):
        continue

    # Find matching class ID
    new_class_id = None
    for key, val in mapping_dict.items():
        if key.lower().replace("_", "") in folder_name.lower().replace("_", ""):
            new_class_id = val
            break

    if new_class_id is None:
        print(f"No matching class ID found for '{folder_name}'. Skipping.")
        continue
    else:
        print(f"Found class ID {new_class_id} for folder '{folder_name}'")

    # Update all .txt files in folder
    txt_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]
    for file_name in txt_files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts:
                parts[0] = str(new_class_id)
                new_lines.append(" ".join(parts) + "\n")

        with open(file_path, "w") as f:
            f.writelines(new_lines)

        print(f"Updated {file_name} in folder '{folder_name}'")

print("All annotation files updated successfully!")
